In [ ]:
# Autoreload modules
%load_ext autoreload
%autoreload 2

In [ ]:
# To have access to moduels
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('')) + '/Modules')

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torchvision.transforms import Compose, Resize

from transformers import ViTConfig, ViTFeatureExtractor, ViTForImageClassification

from matplotlib import pyplot as plt

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from dataloader.dataset import ADNI3Channels
from dataloader.dataloader import ADNILoader

from utils.report import sklearn_classification_report

# Dataset and Dataloader Setup

In [ ]:
image_size = (384, 384)
resize = Resize(size=image_size)

train_transforms = Compose([resize])
valid_transforms = Compose([resize])
test_transforms = Compose([resize])

In [ ]:
train_ds = ADNI3Channels("../Data/Training/", transforms=train_transforms)
valid_ds = ADNI3Channels("../Data/Validation/", transforms=valid_transforms)
test_ds = ADNI3Channels("../Data/Test/", transforms=test_transforms)

In [ ]:
idx = 0
image, label = train_ds[idx]

print("Image shape:", image.shape)
print("Label:", label.item())

print("Number of training samples:", len(train_ds))
print("Number of validation samples:", len(valid_ds))
print("Number of test samples:", len(test_ds), "\n")

fig, axes = plt.subplots(ncols=3, figsize=(6, 2), dpi=300)
for i in range(3):
    axes[i].imshow(image[i, :, :])
    axes[i].axis("off");

In [ ]:
id2label = {0: "CN", 1: "MCI", 2: "AD"}
label2id = {"CN": 0, "MCI": 1, "AD": 2}

print(id2label[label.item()])

In [ ]:
train_batch_size = 10
valid_batch_size = 5
test_batch_size = 5

hparams = {'train_ds': train_ds,
           'valid_ds': valid_ds,
           'test_ds': test_ds,
           'train_batch_size': train_batch_size,
           'valid_batch_size': valid_batch_size,
           'test_batch_size': test_batch_size,
           'num_workers': 20,
           'train_shuffle': False,
           'valid_shuffle': False,
           'test_shuffle': False,
           'train_drop_last': False,
           'valid_drop_last': False,
           'test_drop_last': False,
          }

train_dataloader = ADNILoader(**hparams).train_dataloader()
valid_dataloader= ADNILoader(**hparams).validation_dataloader()
test_dataloader = ADNILoader(**hparams).test_dataloader()

batch = next(iter(train_dataloader))
print(batch[0].shape)
print(batch[1].shape)

# Model

In [ ]:
vit_config = ViTConfig(image_size=image_size,
                       patch_size=32,
                       num_labels=3,
                       output_attentions=True,
                       output_hidden_states=True,
                       hidden_dropout_prob=0.1,
                       # attention_probs_dropout_prob=0.1,
                      )

In [ ]:
class ViT(nn.Module):
    def __init__(self, num_labels=3, return_last_hidden_state=False):
        super(ViT, self).__init__()
        self.return_last_hidden_state = return_last_hidden_state
        self.vit = ViTForImageClassification(vit_config)
        
    def forward(self, x):
        outputs = self.vit(x)
        if self.return_last_hidden_state:
            return outputs.logits, outputs.attentions, outputs.hidden_states
        else:
            return outputs.logits, outputs.attentions

# Saving Hidden States

In [ ]:
device = torch.device('cpu') 
model = ViT(return_last_hidden_state=False).to(device)
feature_extractor = ViTFeatureExtractor(size=image_size,
                                        patch_size=32,
                                        do_resize=False,
                                        do_normalize=False)

model.load_state_dict(torch.load("../ViT/Best models/ViT_loss.pt"))

In [ ]:
def predict(model, dataloader, device):
    y_true = []
    y_pred = []
    
    model.eval()
    with torch.no_grad():
        for step, (x, y) in enumerate(dataloader):
            x = np.split(np.array(x), dataloader.batch_size)
            for i in range(len(x)):
                x[i] = np.squeeze(x[i])
            x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))
            x, y  = x.to(device), y.to(device)
            logits, _ = model(x)
            preds = logits.argmax(1)
        
            y_pred.append(preds.cpu().numpy())
            y_true.append(y.cpu().numpy())

    y_pred = np.concatenate(y_pred, axis=0)
    y_true = np.concatenate(y_true, axis=0)
    
    return y_true, y_pred

y_true, y_pred = predict(model, test_dataloader, device)
sklearn_classification_report(y_true, y_pred)

# Preparing Activation and Label Dataframes

In [ ]:
x_train_df = pd.DataFrame()
y_train_df = pd.DataFrame()

x_test_df = pd.DataFrame()
y_test_df = pd.DataFrame()

model.return_last_hidden_state = True
model.eval()

with torch.no_grad():
    for sample, label in train_dataloader:
        sample = np.split(np.array(sample), train_batch_size)
        for i in range(len(sample)):
            sample[i] = np.squeeze(sample[i])
        sample = torch.tensor(np.stack(feature_extractor(sample)['pixel_values'], axis=0))
        sample, label  = sample.to(device), label.to(device)
        _, _, activation = model(sample)

        act_df = pd.DataFrame(activation[12].reshape((train_batch_size, 145*768)).detach().numpy())
        label_df = pd.DataFrame(label.detach().numpy())

        x_train_df = pd.concat([x_train_df, act_df])
        y_train_df = pd.concat([y_train_df, label_df])

with torch.no_grad():
    for sample, label in test_dataloader:
        sample = np.split(np.array(sample), test_batch_size)
        for i in range(len(sample)):
            sample[i] = np.squeeze(sample[i])
        sample = torch.tensor(np.stack(feature_extractor(sample)['pixel_values'], axis=0))
        sample, label  = sample.to(device), label.to(device)
        _, _, activation = model(sample)

        act_df = pd.DataFrame(activation[12].reshape((test_batch_size, 145*768)).detach().numpy())
        label_df = pd.DataFrame(label.detach().numpy())

        x_test_df = pd.concat([x_test_df, act_df])
        y_test_df = pd.concat([y_test_df, label_df])

# PCA

In [ ]:
x_train_df = x_train_df.reset_index(drop=True)
y_train_df = y_train_df.rename(columns={0: "target"}).reset_index(drop=True)
x_test_df = x_test_df.reset_index(drop=True)
y_test_df = y_test_df.rename(columns={0: "target"}).reset_index(drop=True)

scaler = StandardScaler()
x_train_std = scaler.fit_transform(x_train_df)
x_test_std = scaler.fit_transform(x_test_df)

pca = PCA(n_components=2)
pca_train = pca.fit_transform(x_train_std)
pca_test = pca.fit_transform(x_test_std)

pca_train_df = pd.DataFrame(pca_train, columns = ['principal component 1', 'principal component 2'])
pca_test_df = pd.DataFrame(pca_test, columns = ['principal component 1', 'principal component 2'])

final_train_df = pd.concat([pca_train_df, y_train_df], axis = 1)
final_test_df = pd.concat([pca_test_df, y_test_df], axis = 1)

final_df = pd.concat([final_train_df, final_test_df], axis = 0, keys=['train', 'test'])
final_df

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8), dpi=300)
targets = [0, 1, 2]
colors = ['r', 'g', 'b']

for i in range(len(targets)):
    sample_df = final_df[final_df['target'] == targets[i]]
    
    ax.scatter(sample_df.loc['train', 'principal component 1'],
               sample_df.loc['train', 'principal component 2'],
               s=150,
               alpha=0.3,
               c=colors[i],
               marker='o',
               label=f'{id2label[targets[i]]} (Train)'
              )
    
    ax.scatter(sample_df.loc['test', 'principal component 1'],
               sample_df.loc['test', 'principal component 2'],
               s=150,
               alpha=0.3,
               c=colors[i],
               marker='*',
               label=f'{id2label[targets[i]]} (Test)'
              )
    
ax.legend();
ax.set_xlabel('Principle Component 1')
ax.set_ylabel('Principle Component 2');
# plt.savefig('PCA_ViT.png')